In [ ]:
!pip install pymupdf4llm

In [11]:
import pymupdf4llm

file_name = "tiago_manual.pdf"
md_text = pymupdf4llm.to_markdown(
    doc=file_name,
    embed_images=True,
    # write_images=True,
    image_path=f"images/{file_name}"
)



Processing tiago_manual.pdf...
[                                        ] (0/15[                                        ] (  1/15[                                        ] (  2/15[                                        ] (  3/153[=                                       ] (  4/153[=                                       ] (  5/153[=                                       ] (  6/153[=                                       ] (  7/15[==                                      ] (  8/15[==                                      ] (  9/15[==                                      ] ( 10/15[==                                      ] ( 11/153[===                                     ] ( 12/153[===                                     ] ( 13/153[===                                     ] ( 14/1[===                                     ] ( 15/15[====                                    ] ( 16/15[====                                    ] ( 17/15[====                                    ] ( 18/15[====          

In [ ]:
print(md_text)

In [12]:
import pathlib
pathlib.Path("tiago_text.md").write_bytes(md_text.encode())

24381456

In [26]:
!pip install llama-index


  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 10.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 10.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 9.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 10.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------- ----------- 1.8/2.6 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 8.7 MB/s eta 0:0

In [27]:
from llama_index.readers.file.markdown import MarkdownReader

md_read = MarkdownReader()
md_docs = md_read.load_data(pathlib.Path("tiago_text.md"))


In [32]:
from llama_index.readers.file.pymu_pdf import PyMuPDFReader
pdf_reader = PyMuPDFReader()
pdf_docs = pdf_reader.load_data("tiago_manual.pdf")


In [37]:
pdf_docs[23]


Document(id_='c0fffc99-a79e-4ee9-98f2-6f221c584881', embedding=None, metadata={'total_pages': 153, 'file_path': 'tiago_manual.pdf', 'source': '24'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='OPENING AND CLOSING \n23 \nDoors \nDoor locking / unlocking with key \nDriver door has separate locking facility. \nAll doors can be locked or unlocked from \noutside using the key. \n \nInsert the key and turn it clockwise to \nlock and counter clockwise to open the \ndoor.  \nLocking without a key from inside \n \nAll the doors can also be locked from in-\nside by pressing knob (1) on driver door \nand independently on other doors re-\nspectively. \nOpening the doors from inside \nAll doors can be opened from inside. To \nopen, pull the door opening knob (1) and \nthen lever (2). \n \n \n \n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_templa

In [58]:
for page in pdf_docs:
    print(_get_tables(page.text), len(page.text))
    print(_get_heading(page.text))


None 0

None 30

None 727
CUSTOMER ASSISTANCE
None 1015
3
None 1595
CUSTOMER ASSISTANCE
None 816
CONTENTS
None 651
CONTENTS
None 1449
CONTENTS
None 765
CONTENTS
None 1924
SAFETY
None 1908
SAFETY
None 1491
SAFETY
None 1635
SAFETY
None 1161
SAFETY
None 972
SAFETY
None 1429
SAFETY
None 1468
SAFETY
None 1194
SAFETY
None 1671
SAFETY
None 2053
SAFETY
None 1747
SAFETY
None 1536
OPENING AND CLOSING
None 1852
OPENING AND CLOSING
None 576
OPENING AND CLOSING
None 1320
OPENING AND CLOSING
None 739
OPENING AND CLOSING
None 1027
OPENING AND CLOSING
None 940
OPENING AND CLOSING
None 31
OPENING AND CLOSING
None 577
DASHBOARD AND FEATURES
None 258
DASHBOARD AND FEATURES
None 249
DASHBOARD AND FEATURES
None 1379
DASHBOARD AND FEATURES
None 1070
DASHBOARD AND FEATURES
None 856
DASHBOARD AND FEATURES
None 1072
DASHBOARD AND FEATURES
None 414
DASHBOARD AND FEATURES
None 1779
DASHBOARD AND FEATURES
None 75
DASHBOARD AND FEATURES
None 72
DASHBOARD AND FEATURES
None 836
DASHBOARD AND FEATURES
None 1267
DASHB

In [59]:
print(_get_figures("""

![](data:image/png;base64,iVBOR) """))
# print(pdf_docs[11].text)


['data:image/png;base64,iVBOR']


In [ ]:
for page in pdf_docs:
    print(_get_tables(page.text), len(page.text))


In [46]:
from typing import List
def _split_to_pages(md_text: str) -> List[str]:
    return md_text.split("-----")


def _get_heading(page: str) -> str:
    import re
    heading_pattern = r'^(.*?)(?:\s*#{1,6}\s*)?$'  # Match 1-6 #s for valid markdown headings
    match = re.match(heading_pattern, page.split("\n")[0].strip())
    if match:
        return match.group(1).strip()
    return ""

def _get_figures(page: str) -> List[str]:
    import re
    figure_pattern = r'\!\[.*?\]\((.*?)\)'
    matches = re.findall(figure_pattern, page)
    return matches

def _get_tables(page: str) -> str:
    import re
    table_pattern = r'(\|[^\n]*\|(?:\n\|[^\n]*\|)+)'  # This matches all rows of the table
    match = re.search(table_pattern, page, re.DOTALL)
    if match:
        return match.group(0)
    else:
        return None

pages = _split_to_pages(md_text)
count = 0
for page in pages[14:]:
    heading = _get_heading(page)
    figures = _get_figures(page)
    tables = _get_tables(page)
    print(heading, figures, tables)
    break
    count += 1
    if count > 10:
        break



 [] None


In [20]:
tables

[]